In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
%%capture --no-stderr
!pip install python-dotenv openAI langchain_core langchain_openai langchain langchain-community sentence_transformers rank_bm25

# **오픈AI 임베딩을 활용한 벡터들 간 코사인 유사도 계산**

In [ ]:
# 라이브러리 불러오기
import os
from dotenv import load_dotenv
import numpy as np
from numpy import dot
from numpy.linalg import norm
import pandas as pd
from langchain.embeddings import OpenAIEmbeddings
from langchain_openai import OpenAI

# .env 파일에서 환경 변수 로드
load_dotenv("/content/.env")
# 환경 변수에서 API 키 가져오기
api_key = os.getenv("OPENAI_API_KEY")

In [ ]:
embeddings = OpenAIEmbeddings(model="text-embedding-ada-002")
query_result = embeddings.embed_query("저는 배가 고파요")
print(query_result)

# **오픈AI 임베딩을 활용한 문장들 간 코사인 유사도 계산**

In [ ]:
data = [
    '주식 시장이 급등했어요',
    '시장 물가가 올랐어요',
    '전통 시장에는 다양한 물품들을 팔아요',
    '부동산 시장이 점점 더 복잡해지고 있어요',
    '저는 빠른 비트를 좋아해요',
    '최근 비트코인 가격이 많이 변동했어요',
]

df = pd.DataFrame(data, columns=['text'])
df

In [ ]:
# 텍스트를 임베딩 벡터로 변환하는 함수 정의
def get_embedding(text):
  return embeddings.embed_query(text)

# DataFrame의 각 행에 대해 text 열의 내용을 임베딩 벡터로 변환
# apply는 각 행(row)에 대해 지정한 함수를 적용하는 메서드
# axis=1 → "행 단위로" 적용하겠다는 뜻 (열 단위는 axis=0)

# df['embedding'] = ...
# 새 컬럼 **embedding**을 만들어서 변환된 임베딩 벡터를 저장
df['embedding'] = df.apply(
    lambda row: get_embedding(row.text),
    axis=1
)

df

In [ ]:
# 코사인 유사도 계산 함수
def cos_sim(A, B):
  return dot(A, B)/(norm(A) * norm(B))

# 주어진 쿼리와 가장 유사항 상위 3개의 문서를 반환하는 함수
def return_answer_candidate(df, query):
  # 쿼리 텍스트를 임베딩 벡터로 변환
  query_embedding = get_embedding(query)

  # DataFrame의 각 문서 임베딩과 쿼리 임베딩 간의 유사도 계산

  # array = numpy 배열로 변환
  # NumPy 배열은 벡터·행렬 연산이 바로 가능해서, 임베딩 벡터 다룰 때 필수로 사용
  df["similarity"] = df.embedding.apply(lambda x: cos_sim(np.array(x), np.array(query_embedding)))

  # 유사도가 높은 순으로 정렬하고 상위 3개 문서 선택
  top_three_doc = df.sort_values("similarity", ascending=False).head(3)

  return top_three_doc

# 예시 쿼리로 유사한 문서 검색
sim_result = return_answer_candidate(df, '과일 값이 비싸다')

sim_result

# **BGE-M3 모델 임베딩**

In [ ]:
# 라이브러리 불러오기
from langchain.embeddings import HuggingFaceBgeEmbeddings

# BGE-M3 모델 초기화
embeddings = HuggingFaceBgeEmbeddings(model_name="BAAI/bge-m3")

# 데이터셋 생성
data = [
    '주식 시장이 급등했어요',
    '시장 물가가 올랐어요',
    '전통 시장에는 다양한 물품들을 팔아요',
    '부동산 시장이 점점 더 복잡해지고 있어요',
    '저는 빠른 비트를 좋아해요',
    '최근 비트코인 가격이 많이 변동했어요',
]

df = pd.DataFrame(data, columns=['text'])

# DataFrame의 각 행을벡터로 변환
df['embedding'] = df['text'].apply(get_embedding)

df

In [ ]:
# 쿼리 예시로 유사 문서 검색
sim_result = return_answer_cnadidate(df, '과일 값이 비싸다')
sim_result